# ##2019.07.09 ORM과RE에 대한 이해

ORM에 대한 이해

In [1]:
# E-relationship model의 설계법 -> 최소는 0부터 시작해서 n대n의 관계가 될 수 있다.
# 사람 한명이 여러 답변을 달 수 있어 -> 1대n의 관계를 가지게 된다.

# INSERT INTO question values (?,?,?,?,?) -> question_id = 1
# SELECT tag_id, name FROM tag WHERE name = 'python' ; -> tag_id = 1
# UPDATE tag SET tagged_count = tagged_count + 1 WHERE NAME = 'Python'
# 앞서 question과 tag를 join하면 tag에 어떠한 question이 있는지 볼 수 있다 
# 

In [ ]:
# ORM을 통하여 REGB와 ...

In [7]:
#예제 1
!pip install sqlalchemy

  Stored in directory: C:\Users\lucro\AppData\Local\pip\Cache\wheels\f4\74\97\23789e1e541fb3d64dc9a921a676539695b3d184cf77f5b009
Successfully built sqlalchemy


In [8]:
import sqlalchemy

In [28]:
from sqlalchemy import create_engine

engine = create_engine("sqlite:///:memory:",echo = True)

engine

Engine(sqlite:///:memory:)

In [ ]:
# 접근 준비만 했으뿐 아직 접근하지는 못했음. 

In [12]:
type(engine)

sqlalchemy.engine.base.Engine

In [13]:
# DBS 관계없이 공통된 틀을 만들어 줄 예정 -> class로 만들 수 있음 // int 이외에 데이터 type 만들 수 있음 
# Table 객체, column 객체, meta data를 전부 볼 수 있음

In [16]:
#
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey

In [17]:
metadata = MetaData()

In [19]:
users = Table('users',metadata,
             Column('id',Integer,primary_key = True),
             Column('name',String),
             Column('fullname',String),
             )

In [21]:
addresses = Table('adresses',metadata,
             Column('id',Integer,primary_key = True),
             Column('user_id',None, 
                     ForeignKey('users.id')),
             Column('email_address',String, nullable = False),
             )


InvalidRequestError: Table 'adresses' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.

In [22]:
# metadata.create_all(engine)

In [24]:
# Table들을 성공적으로 생성했다!! // 수정 불가한 상태로 넣어짐
metadata.tables

immutabledict({'users': Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), schema=None), 'adresses': Table('adresses', MetaData(bind=None), Column('id', Integer(), table=<adresses>, primary_key=True, nullable=False), Column('user_id', Integer(), ForeignKey('users.id'), table=<adresses>), Column('email_address', String(), table=<adresses>, nullable=False), schema=None)})

In [29]:
metadata.create_all(engine)

2019-07-09 11:22:21,215 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-07-09 11:22:21,217 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 11:22:21,220 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-07-09 11:22:21,221 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 11:22:21,224 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2019-07-09 11:22:21,225 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 11:22:21,227 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("adresses")
2019-07-09 11:22:21,228 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 11:22:21,230 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2019-07-09 11:22:21,231 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 11:22:21,232 INFO sqlalchemy.engine.base.Engine COMMIT
2019-07-09 11:22:21,234 INFO sqlalchemy.engine.ba

In [30]:
# Table은 immutable이게이 수정이 안됨 -> table를 삭제 -> 다시 만들어서 binding시켜야 한다
# memory는 확인하지 못한다 -> 파일을 다시 떨궈야 함???

In [32]:
# INSERT의 이용방법

insert = users.insert()
print(insert)

insert = users.insert().values(
         name = 'kim', fullname = 'Anonymous, Kim')
print(insert)

insert.compile().params

INSERT INTO users (id, name, fullname) VALUES (:id, :name, :fullname)
INSERT INTO users (name, fullname) VALUES (:name, :fullname)


{'name': 'kim', 'fullname': 'Anonymous, Kim'}

In [34]:
# 우리가 parameters을 두개 넘겼음 -> 그 다음 찍어보니 2개의 parameter만 named 형태로 넘어감 -> 그러나 어디인가 숨겨 있음 
# 이게 dictonary 값으로 넘어가서 -> key&value로 넘어가 있음 -> 따라서 compile을 통해 이를 찾아냄
# **이런 방식으로 값을 집어 넣는 것이 더 쉽고 편의함. 참고로 id는 아직 값을 할당하지 않았음

In [35]:
# result = conn.execute(insert, name = 'Lee', fullname = 'Unknown,Lee') -> 위와 똑같이 값을 insert하는 또 하나의 방법임

In [36]:
conn = engine.connect()
insert.bind = engine
str(insert)
cur = conn.execute(insert)
cur.inserted_primary_key

2019-07-09 11:35:22,078 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname) VALUES (?, ?)
2019-07-09 11:35:22,079 INFO sqlalchemy.engine.base.Engine ('kim', 'Anonymous, Kim')
2019-07-09 11:35:22,081 INFO sqlalchemy.engine.base.Engine COMMIT


[1]

In [45]:
insert = users.insert()
result = conn.execute( insert, 
                       name = "lee",
                        fullname = "unknow, Lee" )
result.inserted_primary_key

2019-07-09 11:42:19,725 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname) VALUES (?, ?)
2019-07-09 11:42:19,727 INFO sqlalchemy.engine.base.Engine ('lee', 'unknow, Lee')
2019-07-09 11:42:19,728 INFO sqlalchemy.engine.base.Engine COMMIT


[4]

In [44]:
# 일단 user id 1,2를 Qmark방식으로 insert한 다음 이를 자동으로 commit하는 거 같음.
conn.execute(addresses.insert(),[
    {"user_id":1,
     "email_address":"mylady@naver.com"},
    {"user_id":2,
     "email_address":"excuseme@nate.com"}
])

2019-07-09 11:40:14,311 INFO sqlalchemy.engine.base.Engine INSERT INTO adresses (user_id, email_address) VALUES (?, ?)
2019-07-09 11:40:14,313 INFO sqlalchemy.engine.base.Engine ((1, 'mylady@naver.com'), (2, 'excuseme@nate.com'))
2019-07-09 11:40:14,314 INFO sqlalchemy.engine.base.Engine COMMIT


In [46]:
# SELECT의 이용방법

In [50]:
# users 모든 query정보를 알고 싶을 떄는 이 같은 방법을 사용한다. 

from sqlalchemy.sql import select

query = select([users])
result = conn.execute(query)

for row in result : 
    print(row)

2019-07-09 11:44:56,114 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2019-07-09 11:44:56,115 INFO sqlalchemy.engine.base.Engine ()
(1, 'kim', 'Anonymous, Kim')
(2, 'lee', 'unknow, Lee')
(3, 'lee', 'unknow, Lee')
(4, 'lee', 'unknow, Lee')


In [53]:
# 이 방법 또한 가능하다
cur = conn.execute(select([users]))
cur.fetchall()
#임시 cursur을 반환하여 이를 프락시로 받는다 

2019-07-09 13:11:05,586 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2019-07-09 13:11:05,587 INFO sqlalchemy.engine.base.Engine ()


[(1, 'kim', 'Anonymous, Kim'),
 (2, 'lee', 'unknow, Lee'),
 (3, 'lee', 'unknow, Lee'),
 (4, 'lee', 'unknow, Lee')]

In [54]:
# Conjunctions 
# 아래의 방법을 통하여 join을 했거나 특정 attribute들을 찾을 수 있음!

In [55]:
from sqlalchemy import and_,or_,not_

In [60]:
print (users.c.id == addresses.c.user_id)
print (users.c.id ==1 )
print ((users.c.id == 1).compile().params)

users.id = adresses.user_id
users.id = :id_1
{'id_1': 1}


In [67]:
result = conn.execute(select([users]).where (users.c.id == 1))

2019-07-09 13:19:22,888 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users 
WHERE users.id = ?
2019-07-09 13:19:22,890 INFO sqlalchemy.engine.base.Engine (1,)


In [68]:
for row in result :
    print(row)

(1, 'kim', 'Anonymous, Kim')


In [70]:
result = conn.execute(select([users,addresses]).where(users.c.id == addresses.c.user_id))

2019-07-09 13:20:47,619 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname, adresses.id, adresses.user_id, adresses.email_address 
FROM users, adresses 
WHERE users.id = adresses.user_id
2019-07-09 13:20:47,620 INFO sqlalchemy.engine.base.Engine ()


In [73]:
# 주로 for문을 이용하여 join한 table을 확인하다
for row in result :
    print(row)

In [74]:
print (users.join(addresses))

print(users.join(addresses, users.c.id == adresses))

users JOIN adresses ON users.id = adresses.user_id


In [ ]:
quary = select([users.c.id,
                users.c.fullname,
               addresses.email_address])

In [84]:
# Table 객체를 METADA로 받아 사용하는 법을 알아보자!

In [79]:
artist = Table ("Artist",metadata,
               Column("id", Integer, Primary_key = True),
               Column ("name",String,nullable=False),
               extend_existing = True)

In [80]:
album = Table ("Album",metadata,
               Column("id", Integer, Primary_key = True),
               Column ("title",String,nullable=False),
               Column ("artist_id",Integer,ForeignKey("Artist.id")),
               extend_existing = True)

In [81]:
genre = Table ("Genre",metadata,
               Column("id", Integer, Primary_key = True),
               Column ("name",String,nullable=False),
               extend_existing = True)

In [83]:
track = Table("Track", metadata,
               Column("id", Integer, Primary_key = True),
               Column ("title",String,nullable=False),
               Column ("length",Integer,nullable=False),
               Column ("rating",Integer,nullable=False),
               Column ("count",Integer,nullable=False),
               Column ("album_id",Integer,ForeignKey("Album.id")),
               Column ("genre_id",Integer,ForeignKey("Genre.id")),
               extend_existing = True)

In [86]:
# Table을 보여주는 법

tables = metadata.tables

for table in tables:
    print(table)

users
adresses
Artist
Album
Genre
Track


In [97]:
metadata.create_all(engine) # memory에서 engine으로 동기화 할 필요가 있음

2019-07-09 14:06:32,236 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2019-07-09 14:06:32,237 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 14:06:32,239 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("adresses")
2019-07-09 14:06:32,239 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 14:06:32,241 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Artist")
2019-07-09 14:06:32,241 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 14:06:32,243 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Album")
2019-07-09 14:06:32,243 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 14:06:32,245 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Genre")
2019-07-09 14:06:32,245 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 14:06:32,248 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Track")
2019-07-09 14:06:32,248 INFO sqlalchemy.engine.base.Engine ()


In [100]:
# Table에 객체들이 저장되어 있는지 확인 하는 방법??
for table in engine.table_names():
    print(table)

2019-07-09 14:06:36,583 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2019-07-09 14:06:36,584 INFO sqlalchemy.engine.base.Engine ()
Album
Artist
Genre
Track
adresses
users


In [101]:
# INSERT 하는 방법

conn.execute(artist.insert(),[
    {"name":"Led"},
    {"name":"Daniel"}
])


conn.execute(album.insert(),[
    {"title":"Hoho", "artist_id":1},
    {"title":"Lulu", "artist_id":2}
])

conn.execute(genre.insert(),[
    {"name":"Rock"},
    {"name":"Metal"}
])

conn.execute(track.insert(),[
    {"title":"Black","rating":5,"length":3234,"count":0, "album_id":1,"genre_id":1},
    {"title":"White","rating":3,"length":1182,"count":0, "album_id":2,"genre_id":2},
    {"title":"FUnkt","rating":7,"length":32,"count":0, "album_id":1,"genre_id":2},
    {"title":"Angds","rating":1,"length":134,"count":0, "album_id":2,"genre_id":1},
])



2019-07-09 14:06:40,524 INFO sqlalchemy.engine.base.Engine INSERT INTO "Artist" (name) VALUES (?)
2019-07-09 14:06:40,525 INFO sqlalchemy.engine.base.Engine (('Led',), ('Daniel',))
2019-07-09 14:06:40,527 INFO sqlalchemy.engine.base.Engine COMMIT
2019-07-09 14:06:40,528 INFO sqlalchemy.engine.base.Engine INSERT INTO "Album" (title, artist_id) VALUES (?, ?)
2019-07-09 14:06:40,529 INFO sqlalchemy.engine.base.Engine (('Hoho', 1), ('Lulu', 2))
2019-07-09 14:06:40,530 INFO sqlalchemy.engine.base.Engine COMMIT
2019-07-09 14:06:40,532 INFO sqlalchemy.engine.base.Engine INSERT INTO "Genre" (name) VALUES (?)
2019-07-09 14:06:40,533 INFO sqlalchemy.engine.base.Engine (('Rock',), ('Metal',))
2019-07-09 14:06:40,534 INFO sqlalchemy.engine.base.Engine COMMIT
2019-07-09 14:06:40,536 INFO sqlalchemy.engine.base.Engine INSERT INTO "Track" (title, length, rating, count, album_id, genre_id) VALUES (?, ?, ?, ?, ?, ?)
2019-07-09 14:06:40,537 INFO sqlalchemy.engine.base.Engine (('Black', 3234, 5, 0, 1, 1)

In [106]:
# where

trackResult = conn.execute(select([track])
                          .where(
                              and_(track.c.album_id == 1, track.c.genre_id == 1)
                          )
                          )
for row in trackResult:
    print(row)

2019-07-09 14:11:37,633 INFO sqlalchemy.engine.base.Engine SELECT "Track".id, "Track".title, "Track".length, "Track".rating, "Track".count, "Track".album_id, "Track".genre_id 
FROM "Track" 
WHERE "Track".album_id = ? AND "Track".genre_id = ?
2019-07-09 14:11:37,634 INFO sqlalchemy.engine.base.Engine (1, 1)
(None, 'Black', 3234, 5, 0, 1, 1)


In [109]:
#join 
# 이런식으로 같이 join을 하고 확인할 수 있음
result = conn.execute(track
                     .select()
                     .select_from(track.join(album)))

for row in result.fetchall():
    print(row)

2019-07-09 14:13:36,087 INFO sqlalchemy.engine.base.Engine SELECT "Track".id, "Track".title, "Track".length, "Track".rating, "Track".count, "Track".album_id, "Track".genre_id 
FROM "Track" JOIN "Album" ON "Album".id = "Track".album_id
2019-07-09 14:13:36,088 INFO sqlalchemy.engine.base.Engine ()


In [114]:
# Multiple Join
# 가져오고 싶은 column만 써주면 된다는 장점이 있음 -> primary key 서로 맞출 필요가 없다는 엄청난 장점이 있음

result = conn.execute(select([track.c.title,album.c.title,genre.c.name,artist.c.name])
                      .select_from(track.join(album).join(genre).join(artist)))

for row in result.fetchall():
    print(row)

2019-07-09 14:17:52,319 INFO sqlalchemy.engine.base.Engine SELECT "Track".title, "Album".title, "Genre".name, "Artist".name 
FROM "Track" JOIN "Album" ON "Album".id = "Track".album_id JOIN "Genre" ON "Genre".id = "Track".genre_id JOIN "Artist" ON "Artist".id = "Album".artist_id
2019-07-09 14:17:52,320 INFO sqlalchemy.engine.base.Engine ()


In [116]:
# 1개의 결과값을 찾았다??

result = conn.execute(select([track.c.title,album.c.title,genre.c.name,artist.c.name])
                      .select_from(track.join(album).join(genre).join(artist))
                      .where(
                      and_(
                      genre.c.id == 1,
                      artist.c.id ==1
                      
                      )))

for row in result.fetchall():
    print(row)

2019-07-09 14:19:00,150 INFO sqlalchemy.engine.base.Engine SELECT "Track".title, "Album".title, "Genre".name, "Artist".name 
FROM "Track" JOIN "Album" ON "Album".id = "Track".album_id JOIN "Genre" ON "Genre".id = "Track".genre_id JOIN "Artist" ON "Artist".id = "Album".artist_id 
WHERE "Genre".id = ? AND "Artist".id = ?
2019-07-09 14:19:00,151 INFO sqlalchemy.engine.base.Engine (1, 1)


In [257]:
# Engine은 살아있는 상태이며, metadata는 이미 비운 상태 ,따라서 아무것도 안나와야 한다
conn.close()
metadata.clear()

In [258]:
# 현재 metadata에는 table이 비여있음
for table in tables:
    print(table)

In [264]:
# Engine에는 table이 존재

for table in engine.table_names():
    print(table)

In [267]:
metadata = MetaData(bind = engine, reflect = True)
metadata.reflect(bind=engine)

# 엔진에서 metadata로 table들을 받아 드렸음 -> 이를 row로 반영
for row in metadata.tables:
    print(row)
                 

c:\users\lucro\anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:1: SADeprecationWarning: The MetaData.reflect flag is deprecated and will be removed in a future release.   Please use the MetaData.reflect() method.
  """Entry point for launching an IPython kernel.


In [242]:
#선언부와 base가 필요하다 -> 이는 상위 class의 개념이 된다.

from sqlalchemy import create_engine
engine = create_engine("sqlite:///:memory:")

from sqlalchemy.ext.declarative import declarative_base
base = declarative_base()

In [243]:
class User(base):
    __tablename__="users"
    
    id = Column(Integer, primary_key = True)
    name = Column(String)
    fullname = Column(String)
    password = Column("password", String)
    
    def __repr__(self):
        return"<T'User(name = '%s', fullname = '%s', password = '%s')>"\
               %(self.name,self.fullname,self.password)

In [244]:
User.__table__

Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), Column('password', String(), table=<users>), schema=None)

In [197]:
base.metadata.create_all(engine)

In [198]:
kim = User(name = "kim", fullname = "annoymou, kim", password="aaa")

print(kim)
print(kim.id)

<T'User(name = 'kim', fullname = 'annoymou, kim', password = 'aaa')>
None


In [199]:
# Session -> database와 우리들의 object를 왔다 갔다 해주는 아이???
# 중간에 ORM이 조율을 해준다 -> 싱크가 틀어질 수 있음 -> 데이터 베이스에는 정보가 없는 경우 , insert가 제대로 되지 않는 경우 

from sqlalchemy.orm import sessionmaker

session = sessionmaker(bind = engine)
session = session()

In [200]:
# commit를 해야 한다
session.add(kim)

In [201]:
session.commit()

In [202]:
kim.password = 'password'

# UPDATE
session.dirty

session.is_modified(kim)

True

In [203]:
for row in session.query(User):
    print(type(row))
    print(row.id,row.name,row.fullname,row.password)

<class '__main__.User'>
1 kim annoymou, kim password


In [206]:
for row in session.query(User.id,User.fullname).filter_by(name = "kim"):
    print(type(row))
    print(row.id,row.fullname)

<class 'sqlalchemy.util._collections.result'>
1 annoymou, kim


In [272]:
sqlalchemy.orm.relationship

<function sqlalchemy.orm.relationship(argument, secondary=None, primaryjoin=None, secondaryjoin=None, foreign_keys=None, uselist=None, order_by=False, backref=None, back_populates=None, post_update=False, cascade=False, extension=None, viewonly=False, lazy='select', collection_class=None, passive_deletes=False, passive_updates=True, remote_side=None, enable_typechecks=True, join_depth=None, comparator_factory=None, single_parent=False, innerjoin=False, distinct_target_key=None, doc=None, active_history=False, cascade_backrefs=True, load_on_pending=False, bake_queries=True, _local_remote_pairs=None, query_class=None, info=None, omit_join=None)>

In [277]:
base.metadata.clear()

In [278]:
from sqlalchemy.orm import relationship 
# back_populates을 하면 table을 join하지 않아도 된다는 장점이 있음 -> 권장!
class Artists(base):
    __tablename__="Artists"
    
    id = Column(Integer, primary_key = True)
    name = Column(String)
    
    albumList = relationship("Album",back_populates = "artist")
    
    
class Album(base):
    __tablename__="Album"
    
    id = Column(Integer, primary_key = True)
    title = Column(String)
    Artist_id = Column(Integer,ForeignKey("Artist.id"))
    
    artist = relationship ("Artist",back_populates="albumList",uselist = False)
    trackList = relationship("Track",back_populates = "album")

class Genre(base):
    __tablename__="Genre"
    
    id = Column(Integer, primary_key = True)
    name = Column(String)
    
    trackList = relationship("Track",back_populates = "genre")
    
class Track(base):
    __tablename__="Track"
    
    id = Column(Integer, primary_key = True)
    title = Column(String)
    length = Column(Integer)
    rating = Column(Integer)
    count = Column(String)
    album_id = Column(Integer,ForeignKey("Album.id"))
    genre_id = Column(Integer,ForeignKey("Genre.id"))
    
    album = relationship ("Album",back_populates = "trackList",uselist = False)
    genre = relationship ("Genre",back_populates = "trackList",uselist = False)

c:\users\lucro\anaconda3\envs\py36\lib\site-packages\sqlalchemy\ext\declarative\clsregistry.py:129: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.Artists, and will be replaced in the string-lookup table.
  % (item.__module__, item.__name__)
c:\users\lucro\anaconda3\envs\py36\lib\site-packages\sqlalchemy\ext\declarative\clsregistry.py:129: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.Album, and will be replaced in the string-lookup table.
  % (item.__module__, item.__name__)
c:\users\lucro\anaconda3\envs\py36\lib\site-packages\sqlalchemy\ext\declarative\clsregistry.py:129: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.Genre, and will be replaced in the string-lookup table.
  % (item.__module__, item.__name__)


In [280]:
#metadata.clear() Table에 객체들이 저장되어 있는지 확인 하는 방법??
for table in engine.table_names():
    print(table)

In [282]:
# base.metadata.create_all(engine)
# base data는 engine에 따로 올릴 필요가 없음

Regular Expression에 대한 이해

In [284]:
# 속도가 빠르다는 특징을 지님 -> text을 처리할때 효율적 
# regular expression?? -> 패턴을 만드는 일환이라 생각하면 편함
# 왜 쓸까?? -> 주어진 text으로 부터 unstructured를 structured으로 바꾸고자 함 , 특정한 패턴을 찾음

In [286]:
# 쓰는 이유?
# 1.) 속도가 매우 빠름 -> linux상 이를 사용하면 매우매우 빠름 
# 2.) NLP에 반드시 이 친구를 써야 한다 -> 한국 text는 이를 쓰는 것이 매우 효과적 -> 한국말 처리가 매우 중요
# ex.) 영어는 하나의 단어이기에 밀어넣으면 쉬움 // 그러나 한국어는 어근을 찾지 못하여 어려움.... 다 다르다네....음절?? 음소?? 음가???
# 3.) 이메일이나 계정 등 validation을 위하여 쓰임 -> 매우 빨라야함, 우리나라는 1초 걸려야함 -> validation이 중요
# 단점 -> learning curve가 존재 -> 익숙해지기 위해서는 매우 오래 걸린다는 단점이 있음

In [ ]:
# Anchor : ^$
# Alternative : |
# Gropuing: 0
# Quantification: ?+*{m,n}
# Main Characters : .[][-][^]
# Character Classes : \w\d\s\w....

In [ ]:
# .(dot) 
#   - a,b 
#   - a + 모든 문자 + b
#   - aab,a)b

# *(asterisk)
# - ab*c
# - a + b(0번 이상 반복) + c
# - ac,abc,abbbbbbbbc

# +(plus)
# - ab+c
# - a+b(1번 이상 반복)
# - abc,abbbbbbbc

# {n|min,|min,max}(curly braces)
# - {0,}=*,{1,}=+
# ab{1}c, ab{2,6}c

# ?(question)
# - ab?c b{0,1}
# - a+b(1개 있거나, 없거나) + c
# ac,abc

In [293]:
# 예제
import re
content = "hello world"

print(re.match("w",content))
print(re.search("w",content))


None
<_sre.SRE_Match object; span=(6, 7), match='w'>


In [297]:
# 정규식을 이용하면 확실히 더 편해짐

data = """park 800905-1049118
kim  700905-1059119  """

pat = re.compile("(\d{6})[-]\d{7}")
print(pat.sub("\g<1>-*******",data))

park 800905-*******
kim  700905-*******  


In [299]:
p = re.compile('Crow|Servo')
m = p.match('CrowHello')

print(m)

<_sre.SRE_Match object; span=(0, 4), match='Crow'>


In [300]:
# Class를 찾아달라고 한다

p = re.compile(r'\bclass\b')

print(p.search('no class at all'))
print(p.search(' one subclass is'))
print(p.search('the disclassfied algorithm'))
      

<_sre.SRE_Match object; span=(3, 8), match='class'>
None
None


In [302]:
# Group에 대한 예제

m = re.match(r"(\w+)(\w+)","Isac Newton, psychologist")
print(m.group(0))
print(m.group(1))
print(m.group(2))
print(m.group(1,2))

Isac
Isa
c
('Isa', 'c')


In [305]:
p = re.compile(r"(\w+) (\w+)")
m = p.search("Isac Newton, psychologist")
print(m.group())

p.sub("\g<2> \g<1>","Isac Newton, psychologist")

Isac Newton


'Newton Isac, psychologist'